In [ ]:
!pip install Tashaphyne
!pip install Unidecode
!pip install aiogoogletrans
!pip install cltk
!pip install bert-tokenizer
!pip install transformers


from transformers import BertTokenizer, BertForNextSentencePrediction, BertForSequenceClassification
from tqdm.notebook import tqdm
import torch


from datetime import datetime
from google.colab import files
from datetime import datetime
from google.colab import drive

import nltk
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer
from tashaphyne.stemming import ArabicLightStemmer
from __future__ import unicode_literals
from unidecode import unidecode
from textblob import TextBlob
from aiogoogletrans import Translator
import asyncio
import unicodedata

from cltk.corpus.utils.importer import CorpusImporter
from cltk.corpus.arabic.alphabet import *
from cltk.stop.arabic.stopword_filter import stopwords_filter as ar_stop_filter
from cltk.tokenize.word import WordTokenizer



import pandas as pd
import numpy as np
import pickle
import time
import json
import glob
import re
import os

import warnings
warnings.simplefilter("ignore")
nltk.download('stopwords')
nltk.download('punkt')
drive.mount('/content/drive')

     |████████████████████████████████| 245kB 5.1MB/s 
     |████████████████████████████████| 112kB 6.4MB/s 
  Created wheel for pyarabic: filename=PyArabic-0.6.10-cp36-none-any.whl size=113324 sha256=7ff63bbbc48a1838da945b39ac8ef93ac343fd86a7f4c9852af4e99b794d4d95
  Stored in directory: /root/.cache/pip/wheels/10/b8/f5/b7c1a50e6efb83544844f165a9b134afe7292585465e29b61d
Successfully built pyarabic
     |████████████████████████████████| 245kB 6.1MB/s 
     |████████████████████████████████| 1.3MB 4.1MB/s 
     |████████████████████████████████| 143kB 13.0MB/s 
     |████████████████████████████████| 296kB 12.3MB/s 
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3163 sha256=5dc269ce4820383ce73762bdf833fd742b4c8600d8400c01474a2c8303d54646
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built idna-ssl
     |████████████████████████████████| 634kB 4.3MB/s 
     |██████████████████████████

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/beforSplitedDataset/tweets.csv',sep=',',names=['ID','Tweet','Class']) 


In [ ]:
print(len(data))
data.tail()


80848


,ID,Tweet,Class
80843,873004402113695747,حمد لهه ونو وثق شغل جدو حول طلع حسن عند حول طو...,trust
80844,389849012956057600,ونو وكد تمم تحد زود جنن ضبط بلس د حول هتف جمهر...,trust
80845,884428772866498560,ونو وكد سلف قطر ٪ جيش صرصر,trust
80846,716683864257781760,زين ل دقق نخب دخل هدف شغل ثون ونو وثق ونك شفت ...,trust
80847,449178499962908673,دون دحح لوم ونو وثق فكر,trust


In [ ]:
data['Class'].value_counts()

happiness       22847
trust           13416
sadness         12937
anticipation    11806
fear            11214
surprise         6691
anger            1161
disgust           775
Class               1
Name: Class, dtype: int64

In [ ]:
label_dict={'happiness':0, 'trust' :1 , 'sadness': 2 ,
            'anticipation':3 ,'fear' :4 ,'surprise': 5 ,
            'anger' : 6,  'disgust':7}
data['label'] = data.Class.replace(label_dict)
data.Class.values

array(['Class', 'anger', 'anger', ..., 'trust', 'trust', 'trust'],
      dtype=object)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(data.index.values[1:], 
                                                  data.Class.values[1:], 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=data.Class.values[1:])

data['data_type'] = ['not_set']*data.shape[0]

data.loc[X_train, 'data_type'] = 'train'
data.loc[X_val, 'data_type'] = 'val'

data.groupby(['Class', 'label', 'data_type']).count()

ID  Tweet
Class        label data_type              
Class        Class not_set        1      1
anger        6     train        987    987
                   val          174    174
anticipation 3     train      10035  10035
                   val         1771   1771
disgust      7     train        659    659
                   val          116    116
fear         4     train       9532   9532
                   val         1682   1682
happiness    0     train      19420  19420
                   val         3427   3427
sadness      2     train      10996  10996
                   val         1941   1941
surprise     5     train       5687   5687
                   val         1004   1004
trust        1     train      11403  11403
                   val         2013   2013

In [ ]:
from torch.utils.data import TensorDataset

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

                                          
encoded_data_train = tokenizer.batch_encode_plus(
    data[data.data_type=='train'].Tweet.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    truncation=True,
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    data[data.data_type=='val'].Tweet.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    truncation=True,
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(np.array([data[data.data_type=='train'].label.values.astype(int)]))

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(np.array([data[data.data_type=='val'].label.values.astype(int)]))


dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train.T)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val.T)

In [ ]:
labels_train.T

tensor([[6],
        [6],
        [6],
        ...,
        [1],
        [1],
        [1]])

In [ ]:
input_ids_train


tensor([[  101,  1280, 15394,  ...,     0,     0,     0],
        [  101,  1298, 29833,  ...,     0,     0,     0],
        [  101,  1293, 17149,  ...,     0,     0,     0],
        ...,
        [  101,  1298, 15915,  ...,     0,     0,     0],
        [  101,  1298, 15915,  ...,     0,     0,     0],
        [  101,  1278, 29836,  ...,     0,     0,     0]])

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 10

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [ ]:
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cpu


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:

for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'/content/drive/MyDrive/beforSplitedDataset/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

In [ ]:
model.load_state_dict(torch.load('data_volume/finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))


In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)


In [ ]:
accuracy_per_class(predictions, true_vals)
